In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from mne.io import RawArray
from mne import create_info, EpochsArray
from mne_icalabel import label_components
from mne.decoding import Scaler
import os

In [2]:
folder_path = "dataset/Crowdsource processed data/"
files = os.listdir(folder_path)

In [3]:
files

['Alpha Supression_EPOCPLUS_2020_11_10T12_58_downsampled_output.csv',
 'Alpha Supression_EPOCPLUS_2020_11_12T15_07_downsampled_output.csv',
 'Alpha Supression_EPOCPLUS_2021_02_28T14_03_downsampled_output.csv',
 'Alpha Supression_EPOCPLUS_2021_02_28T15_27_downsampled_output.csv',
 'Alpha Supression_EPOCPLUS_2021_03_01T16_57_downsampled_output.csv',
 'Alpha Supression_EPOCPLUS_2021_03_01T21_53_downsampled_output.csv',
 'Alpha Supression_EPOCX_2021_02_28T22_30_downsampled_output.csv',
 'Alpha Supression_EPOCX_2021_03_12T22_45_downsampled_output.csv',
 'Alpha Supression_EPOCX_2021_03_19T02_12_downsampled_output.csv',
 'Alpha Supression_EPOCX_2021_04_06T14_46_downsampled_output.csv',
 'Alpha Supression_EPOCX_2021_07_10T13_32_downsampled_output.csv',
 'Alpha Supression_EPOCX_2021_07_17T20_44_downsampled_output.csv',
 'Alpha Supression_EPOC_2020_11_27T22_20_downsampled_output.csv',
 'Alpha Supression_INSIGHT_2020_11_23T22_08_downsampled_output.csv',
 'Alpha Supression_INSIGHT_2020_12_02T12_24

In [4]:
df_list = []
y = []

for file in files:
    df = pd.read_csv(folder_path + file)
    eyesopen_df = df[df["marker_label"] == "eyesopen_element"]
    eyesclose_df = df[df["marker_label"] == "eyesclose_element"]
    if len(df.columns) > 14 and len(eyesopen_df) >= 15360 and len(eyesopen_df) >= 15360:
        df_list.append(eyesopen_df.iloc[:15360])
        y.append(1)
        df_list.append(eyesclose_df.iloc[:15360])
        y.append(0)
        
y = np.array(y)

C:\Users\sh204\AppData\Local\Temp\ipykernel_23172\2202653997.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path + file)


In [5]:
channels = df_list[0].columns
channels = channels[:-1]
eeg_channels = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]
info = create_info(ch_names=eeg_channels, sfreq=128, ch_types="eeg")
montage = mne.channels.make_standard_montage('standard_1020')

raws = []
X = []
for df in df_list:
    x = df.loc[:, channels].to_numpy()
    x = np.transpose(x, (1, 0))
    # X.append(x)
    
    # x_fft = np.fft.rfft(x.copy(), axis=-1)  
    # time = np.arange(x_fft.shape[-1])
    # freq = np.fft.rfftfreq(len(time), d=1/128)
    # x_fft[:, np.where(freq < 1)] = 0
    # x_fft[:, np.where(freq > 100)] = 0
    # x_ifft = np.fft.irfft(x_fft)

    # raw = RawArray(x_ifft.copy(), info=info, verbose=0)
    raw = RawArray(x.copy(), info=info, verbose=0)
    raw.set_montage(montage)
    raw.set_eeg_reference("average", verbose=0)
    raws.append(raw)
    
# X = np.array(X)

In [6]:
X = np.vstack([[raws[i].copy().get_data() for i in range(len(raws))]])

In [7]:
for i in range(len(raws)):
    raws[i].filter(l_freq=1, h_freq=63.9, verbose=0)
    raws[i].set_eeg_reference("average", verbose=0)

In [8]:
mus_inds_list = []
ica_list = []
ic_labels_list = []
components_flag_dict = {"eye blink": 0, "heart beat": 0, "muscle artifact": 0, "line noise": 0, 
                        "channel noise": 0, "other": 0}

for raw in raws:
    ica = mne.preprocessing.ICA(n_components=13, random_state=97, method="infomax", fit_params=dict(extended=True))
    ica.fit(raw, verbose=0)
    ica_list.append(ica)
    mus_inds, mus_scores = ica.find_bads_muscle(raw, verbose=0)
    mus_inds_list.append(mus_inds)
    ic_labels = label_components(raw, ica, method="iclabel")
    ic_labels_list.append(ic_labels["labels"])
    for key in components_flag_dict.keys():
        if key in ic_labels["labels"] and key != "muscle artifact":
            components_flag_dict[key] += 1
    if len(mus_inds) > 0:
        components_flag_dict["muscle artifact"] += 1

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\sh204\AppData\Local\Temp\ipykernel_23172\3998698722.py:13: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\sh204\AppData\Local\Temp\ipykernel_23172\3998698722.py:13: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

C:\Users\sh204\AppData\Local\Temp\ipykernel_23172\3998698722.py:13: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\sh204\AppData\Local\Temp\ipykernel_23172\3998698722.py:13: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\sh204\AppData\Local\Temp\ipykernel_23172\3998698722.py:13: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


In [9]:
components_flag_dict

{'eye blink': 18,
 'heart beat': 3,
 'muscle artifact': 23,
 'line noise': 0,
 'channel noise': 3,
 'other': 23}

In [10]:
all_data = []
for i in range(len(ic_labels_list)):
    labels = ic_labels_list[i]
    
    # eye artifact removal
    eye_inds = [j for j, label in enumerate(labels) if labels == "eye blink"]
    ica_list[i].exclude = eye_inds
    raw_eye_removed = ica_list[i].apply(raws[i].copy(), verbose=0)
    x_filter_eye_removed = raw_eye_removed.get_data()

    # # heart artifact removal
    # heart_inds = [j for j, label in enumerate(labels) if labels == "heart beat"]
    # ica_list[i].exclude = heart_inds
    # raw_heart_removed = ica_list[i].apply(raws[i].copy(), verbose=0)
    # x_filter_heart_removed = raw_heart_removed.get_data()
    
    # muscle artifact removal
    mus_inds = mus_inds_list[i]
    ica_list[i].exclude = mus_inds
    raw_muscle_removed = ica_list[i].apply(raws[i].copy(), verbose=0)
    x_filter_muscle_removed = raw_muscle_removed.get_data()

    # # line noise removal
    # line_inds = [j for j, label in enumerate(labels) if labels == "line noise"]
    # ica_list[i].exclude = line_inds
    # raw_line_removed = ica_list[i].apply(raws[i].copy(), verbose=0)
    # x_filter_line_removed = raw_line_removed.get_data()

    # # channel noise removal
    # channel_inds = [j for j, label in enumerate(labels) if labels == "channel noise"]
    # ica_list[i].exclude = channel_inds
    # raw_channel_removed = ica_list[i].apply(raws[i].copy(), verbose=0)
    # x_filter_channel_removed = raw_channel_removed.get_data()

    # all removal
    ica_list[i].exclude = list(set(eye_inds + mus_inds))
    raw_all_removed = ica_list[i].apply(raws[i].copy(), verbose=0)
    x_filter_all_removed = raw_all_removed.get_data()

    x = X[i].copy()
    x_filter = raws[i].copy().get_data()
    all_data.append(np.vstack([[x, x_filter, x_filter_eye_removed, x_filter_muscle_removed, x_filter_all_removed]]))

In [11]:
all_data = np.array(all_data)

In [12]:
all_data.shape

(24, 5, 14, 15360)

In [13]:
np.savez('dataset/Crowdsource_filter_artifact_removal.npz', 
         x_all=all_data, y_all=y
        )

x_all:  
(24, 5, 14, 15360)  
(被験者数*2(eyesopen, eyesclose) アウトプットの種類, チャンネル数, 時間ステップ (2min, s_freq=128Hz))  
アウトプットの種類について：  
1. x
2. x_filter
3. x_filter_eye
4. x_filter_muscle
5. x_filter_all

y_all:  
(90, )  
(被験者数*2(eyesopen, eyesclose))